In [1]:
!pip install dill

В качестве данных берем пример из урока 3 с кардио-данными.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import dill

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G' -O train_case2.csv

--2023-05-28 16:32:54--  https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G
Распознаётся drive.google.com (drive.google.com)… 64.233.161.194, 2a00:1450:4010:c01::c2
Подключение к drive.google.com (drive.google.com)|64.233.161.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lar08fteolh1ifqc2bimmhd6hj1afl6u/1685280750000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download&uuid=e6a19c12-5e0e-4f66-8555-aff606d3a6b4 [переход]
Предупреждение: в HTTP маски не поддерживаются.
--2023-05-28 16:32:56--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lar08fteolh1ifqc2bimmhd6hj1afl6u/1685280750000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download&uuid=e6a19c12-5e0e-4f66-8555-aff606d3a6b4
Распознаётся doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.

In [4]:
df = pd.read_csv('train_case2.csv', sep=';')
df.head(5)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


#### Обучение простейшей модели логистической регрессии

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
# разделим данные на train/test
feature = df.drop(columns='cardio')
target = df['cardio']
X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler

# непрерывные признаки
continuos_cols = ['age', 'height', 'weight']

# категориальные признаки
cat_cols = ['gender', 'cholesterol']

# булевы признаки
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
# for cat_col in cat_cols:
#     cat_transformer = Pipeline([
#                 ('selector', ColumnSelector(key=cat_col)),
#                 ('ohe', OHEEncoder(key=cat_col))
#             ])
#     cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [8]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
model = Pipeline([('feats', feats), ('classifier', LogisticRegression())])
model.fit(X_train, y_train)

Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                           

In [9]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(model, f)

#### Предполагаем что модель разворачивается другим человеком отдельно от разработчика модели. Поднимаем модель на сервере Flask

In [10]:
#from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [11]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

        
    request_json = request.get_json()
    
    
    age = request_json["age"]    
    height = request_json["height"]    
    weight = request_json["weight"]   
    gluc = request_json["gluc"]
    smoke = request_json["smoke"]
    alco = request_json["alco"]
    active = request_json["active"]
        
    preds = model.predict_proba(pd.DataFrame({"age": [age],
                                              "height": [height],
                                              "weight": [weight],
                                              "gluc": [gluc],
                                              "smoke": [smoke],
                                              "alco": [alco],
                                              "active": [active]}))
    data["predictions"] = preds[:, 1][0]    
        
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/May/2023 16:38:11] "POST //predict HTTP/1.1" 200 -


OK
